In [1]:
import os
import pandas as pd
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *
from arcpy.analysis import *
import numpy as np

In [2]:
arcpy.da.TableToNumPyArray

In [3]:
# tool function
# 创建渔网
def create_net(output_file,extent_file,cell_size):
    """参数说明：
        output_file: 输出文件
        extent_file: 输入范围文件
        cell_size: 生成的cell大小
    """
    proj = arcpy.Describe(extent_file).spatialReference
    extent_x_min = arcpy.Describe(extent_file).extent.XMin
    extent_y_min = arcpy.Describe(extent_file).extent.YMin
    extent_x_max = arcpy.Describe(extent_file).extent.XMax
    extent_y_max = arcpy.Describe(extent_file).extent.YMax
    with arcpy.EnvManager(outputCoordinateSystem = proj):
        arcpy.management.CreateFishnet(out_feature_class=output_file,origin_coord=f"{extent_x_min} {extent_y_min}",y_axis_coord=f"{extent_x_min} {extent_y_min+10}",cell_width=cell_size,cell_height=cell_size,number_rows=None,number_columns=None,corner_coord=f"{extent_x_max} {extent_y_max}",labels="NO_LABELS",template="#",geometry_type="POLYGON")
# 面转点
def polygon_point(in_feature, out_feature):
    """参数说明：
        in_feature: 输入面
        out_feature: 输出点
    """
    proj = arcpy.Describe(in_feature).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(in_features=in_feature,out_feature_class=out_feature,point_location="INSIDE")

# 筛选点

def select_point(input_polygon,input_point,output_point):
    """参数说明：
        input_polygon: 选择面
        input_point: 待选择点
        output_point: 选择点
    """
    polygon_layer = "polygonLayer"
    point_layer = "pointLayer"
    arcpy.MakeFeatureLayer_management(input_polygon, polygon_layer)
    arcpy.MakeFeatureLayer_management(input_point, point_layer)
    arcpy.SelectLayerByLocation_management(in_layer=point_layer, overlap_type="INTERSECT", select_features=polygon_layer,search_distance=0,selection_type='NEW_SELECTION',invert_spatial_relationship="NOT_INVERT")
    # 导出选择的要素
    arcpy.CopyFeatures_management(point_layer, output_point)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)
# 删除要素
def delete_feature(input_feature):
    arcpy.Delete_management(input_feature)
# 空间连接
def perform_spatial_join(target_layer_path, join_layer_path, output_layer_path, field_mapping_dict):
    """参数说明：
        target_layer_path: 目标图层路径
        join_layer_path: 连接图层路径
        output_layer_path: 输出图层路径
        field_mapping_dict: 字段映射字典
    """
    # 创建 FieldMappings 对象
    field_mappings = arcpy.FieldMappings()

    # 为每个字段对创建 FieldMap 并添加到 FieldMappings
    for target_field, join_field in field_mapping_dict.items():
        field_map = arcpy.FieldMap()
        
        # 添加目标字段和连接字段到FieldMap
        field_map.addInputField(target_layer_path, target_field)
        field_map.addInputField(join_layer_path, join_field)
        
        # 设置输出字段属性
        output_field = field_map.outputField
        output_field.name = target_field
        output_field.aliasName = target_field
        field_map.outputField = output_field

        # 将 FieldMap 添加到 FieldMappings
        field_mappings.addFieldMap(field_map)

    # 执行空间连接操作
    arcpy.analysis.SpatialJoin(target_layer_path, join_layer_path, output_layer_path,"JOIN_ONE_TO_ONE", "KEEP_ALL", field_mappings)


# 点采样
def sample_point(point_,raster_,out_name):
    """根据栅格采样点,输出为表格"""
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    """参数说明：
        table_: 输入数据表
        out_path: 输出路径
        out_name: 输出表名"""
    TableToTable(table_,out_path,out_name)
    return None

# 要素转点
def feature_to_point(input_layer,output_layer):
    """参数说明："""
    proj = arcpy.Describe(input_layer).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(input_layer, output_layer, "INSIDE")

# 删除多余字段
def delete_extra_fields(input_feature_class, fields_list):
    """参数说明："""
    fields = arcpy.ListFields(input_feature_class)
    for field in fields:
        name = field.name
        type = field.type.upper()
        # 如果字段名不在删选列表中且类型不为OID和Geometry便删除这个字段
        if name not in fields_list and type not in ['OID','GEOMETRY']:
            arcpy.DeleteField_management(input_feature_class, [name])

In [4]:
# 设置工作空间
env.workspace = r'D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features'
env.overwriteOutput=True

In [5]:
def create_point(input_layer):
    """生成虚拟点"""
    # 定义文件名称
    fish_net_name = "dunyun_grid"
    net_to_point_name = "dunyun_inside_point"
    select_point_name = "select_point"
    result_point_name = "join_point"
    # 渔网构建
    create_net(fish_net_name,input_layer,300)
    # 要素转点
    polygon_point(fish_net_name,net_to_point_name)
    # 按位置选择
    select_point(input_layer,net_to_point_name,select_point_name)
    # 添加字段
    field_list = ['TL','YL','TS','TZ']
    for one_field in field_list:
        # 添加字段
        add_field(input_table=select_point_name,field_name=one_field,field_type='TEXT')
    # 空间连接赋予地类属性
    # 定义目标要素图层和连接要素图层的路径
    fields_mapping = {
        "TL": "修改代号",
        "YL": "修改代号",
        "TS": "修改代号",
        "TZ": "修改代号"
    }
    # 空间连接
    perform_spatial_join(select_point_name,input_layer,result_point_name,fields_mapping)
    # 删除多余数据
    for one_feature in [fish_net_name,net_to_point_name,select_point_name]:
        delete_feature(one_feature)

In [5]:
# 生成渔网点
create_point(input_layer="dy_tl_dh")

In [6]:
# 生成面内部点
feature_to_point(input_layer="dy_tl_dh",output_layer="dy_tl_dh_point")

In [8]:
# 内部点字段整理
# 添加字段
input_feature = "dy_tl_dh_point"
field_list = ['TL','YL','TS','TZ']
calc_field = "修改代号"
for one_field in field_list:
    # 添加字段
    add_field(input_table=input_feature,field_name=one_field,field_type='TEXT')
    # 注意：这里使用了Python表达式
    arcpy.CalculateField_management(input_feature, one_field, f"!{calc_field}!", "PYTHON3")
# 删除多余字段
delete_extra_fields(input_feature, field_list)


In [6]:
# 渔网点字段整理
# 添加字段
input_feature = "join_point"
field_list = ['TL','YL','TS','TZ']
# 删除多余字段
delete_extra_fields(input_feature, field_list)

In [9]:
# 为两个点位数据增加label
feature_1 = "join_point"
feature_2 = "dy_tl_dh_point"
for one_feature in [[feature_1,'fish_net'],[feature_2,'inner']]:
    features = one_feature[0]
    labels = one_feature[1]
    # 添加字段
    add_field(input_table=features,field_name="label",field_type='TEXT')
    # 注意：这里使用了Python表达式
    arcpy.CalculateField_management(features, "label", f"{labels}", "PYTHON3")


NameError: name 'fish_net' is not defined

ExecuteError: ERROR 999999: 异常错误导致工具失败。 请联系 Esri 技术支持 (http://esriurl.com/support) 以报告漏洞，并参阅错误帮助以获取潜在解决方案或解决方法。
执行(CalculateField)失败。


In [ ]:
# 合并点位数据



# 采样环节

In [3]:
# 对特征因子采样
# result table name
sample_csv_name = 'dy_frequency_dz_dl.csv'
# 选择用于采样的特征因子数据库
stander_raster_gdb = r"D:\ArcGISProjects\workspace\shbyq\features_data_dy.gdb"
env.workspace = stander_raster_gdb # 切换工作空间用于采样
# 用于采样的特征因子列表
feature_list = arcpy.ListRasters()
len(feature_list)

37

In [4]:
# 选择用于采样的要素类
point_data = r"D:\ArcGISProjects\workspace\shbyq\features_data_vector.gdb\features\join_point"

# 用于存放采样表的数据库路径
sample_gdb_path = r"F:\cache_data\frequency_filter\dy"

In [5]:
# 采样点数据名称
sample_name =os.path.basename(point_data)
filed_list = [_.name for _ in arcpy.ListFields(point_data)]
print(filed_list)
# 需要保留的字段
# elements_yes = ['OBJECTID', 'TL','YL','TS','TZ']
elements_yes = ['OBJECTID', 'TZ']
filter_list = [_ for _ in filed_list if _ in elements_yes]
print(filter_list)


['OBJECTID', 'Shape', 'Join_Count', 'TARGET_FID', 'TL', 'YL', 'TS', 'TZ']
['OBJECTID', 'TZ']


In [12]:
# 使用Delete_management函数删除数据库中的所有内容
try:
    arcpy.Delete_management(os.path.join(sample_gdb_path,'tabledata.gdb'))
except:
    pass
# 再创建一个新的数据库
arcpy.management.CreateFileGDB(sample_gdb_path, "tabledata", "CURRENT")

<Result 'F:\\cache_data\\frequency_filter\\dy\\tabledata.gdb'>

In [13]:
# 逐个采样并保存到csv文件
gdb_path = os.path.join(sample_gdb_path,"tabledata.gdb")
for one_feature in feature_list:
    print(one_feature)
    sample_point(point_data,one_feature,os.path.join(gdb_path,one_feature))

DEM
AnalyticalHillshading
Aspect
ChannelNetworkBaseLevel
ChannelNetworkDistance
ClosedDepressions
ConvergenceIndex
LSFactor
PlanCurvature
ProfileCurvature
RelativeSlopePosition
Slope
TopographicWetnessIndex
TotalCatchmentArea
ValleyDepth
Contrast
Correlation
Dissimilarity
Entropy
Homogeneity
Mean
ndvi
PCA_0
PCA_1
SecondMoment
Variance
PRE
SRA
TMP
VAP
WIND
BIO
LON
LAT
PH
DZ
DL


In [33]:
# 导出CSV表用于分析
env.workspace = gdb_path # 切换工作空间用于导出csv文件

In [34]:
# 读取数据表并保存到csv文件
result_df = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(point_data,filter_list))
result_df.rename(columns={"OBJECTID":sample_name},inplace=True)

In [35]:
#  读取每个表的最后一个字段的数据,存储每个表的最后一个字段的数据
for table in feature_list:
    print(table)
    # 将表转换为pandas数据帧
    # df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*"))  # 确保数据表中无空值
    # df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*",skip_nulls=False,null_value=99))  # 确保数据表中无空值
    df = pd.DataFrame(arcpy.da.TableToNumPyArray(table, "*",null_value=0))  # 确保数据表中无空值
    # 提取最后一个字段的数据
    merged_df = df[[sample_name, df.columns[-1]]]
    # 合并
    result_df = pd.merge(result_df, merged_df, on=[sample_name])
# 保存到csv文件
result_df.rename(columns=dict(zip(result_df.columns[-len(feature_list):], feature_list)),inplace=True)
result_df.drop(result_df.columns[0],axis=1,inplace=True)
# result_df.rename(columns={"PH":"PH_T"},inplace=True)
result_df.to_csv(os.path.join(sample_gdb_path,sample_csv_name),index=False,encoding='utf-8')

DEM
AnalyticalHillshading
Aspect
ChannelNetworkBaseLevel
ChannelNetworkDistance
ClosedDepressions
ConvergenceIndex
LSFactor
PlanCurvature
ProfileCurvature
RelativeSlopePosition
Slope
TopographicWetnessIndex
TotalCatchmentArea
ValleyDepth
Contrast
Correlation
Dissimilarity
Entropy
Homogeneity
Mean
ndvi
PCA_0
PCA_1
SecondMoment
Variance
PRE
SRA
TMP
VAP
WIND
BIO
LON
LAT
PH
DZ
DL


In [36]:
df = pd.read_csv(os.path.join(sample_gdb_path,sample_csv_name))
df

,TZ,DEM,AnalyticalHillshading,Aspect,ChannelNetworkBaseLevel,ChannelNetworkDistance,ClosedDepressions,ConvergenceIndex,LSFactor,PlanCurvature,...,SRA,TMP,VAP,WIND,BIO,LON,LAT,PH,DZ,DL
0,13d,961.5000,0.706572,0.231111,827.27014,134.229860,0.0,16.303608,1.840937,0.024353,...,13154.333,15.766666,1.555833,1.658333,194.86105,443245.0,2861300.0,5.355657,9,7
1,106,888.4000,0.426612,6.042653,870.42035,17.979675,0.0,2.823421,13.958138,0.009785,...,13154.333,15.766666,1.555833,1.658333,194.86105,443545.0,2861300.0,5.901145,9,7
2,13d,870.8000,1.131889,1.264116,835.30994,35.490050,0.0,-2.361449,18.948938,-0.010550,...,13154.333,15.766666,1.555833,1.658333,194.86105,443845.0,2861300.0,5.594585,9,3
3,13d,1021.6000,0.186124,5.645218,940.00836,81.591610,0.0,4.197873,13.658210,0.010529,...,13140.667,15.600000,1.540833,1.683333,195.54823,444445.0,2861300.0,5.476191,9,7
4,14d,853.4000,0.811739,4.231506,814.98510,38.414917,0.0,1.524923,20.288977,0.010902,...,13150.667,15.750000,1.551667,1.675000,194.79027,442045.0,2861600.0,5.976630,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25293,14d,1185.2999,0.716325,0.344991,1137.37800,47.921875,0.0,1.584722,13.240182,0.005544,...,12810.917,14.716667,1.446667,1.758333,189.39268,448645.0,2924000.0,5.716153,3,0
25294,14d,1104.7000,0.961624,1.333926,1085.43420,19.265747,0.0,4.382226,7.424496,0.006824,...,12810.917,14.716667,1.446667,1.758333,189.39268,448945.0,2924000.0,5.714003,3,0
25295,14d,1070.0000,0.929952,3.877045,1041.28740,28.712646,0.0,0.931178,23.099926,0.004586,...,12810.917,14.716667,1.446667,1.758333,189.39268,449245.0,2924000.0,5.724402,3,0
25296,14d,1104.7000,1.074531,2.214330,1055.28220,49.417725,0.0,-0.791134,15.086410,-0.001959,...,12832.167,14.908333,1.464167,1.750000,189.22100,450445.0,2924000.0,5.685499,3,0
